# MANAGED / ABSTRACTED RAG USING THE GOOGLE GEMINI FILESEARCH API

- https://blog.google/technology/developers/file-search-gemini-api/


# Verify kernel selection

In [1]:
# Run in a notebook cell and check that it gives the same output as `which python` in terminal
import sys
print(sys.executable)

/Users/zubinpratap/Documents/my_code/research-public/.venv/bin/python


# Install the libraries


In [2]:
!uv pip install -U google-genai dotenv


Using Python 3.13.5 environment at: /Users/zubinpratap/Documents/my_code/research-public/.venv
Resolved 25 packages in 156ms                                        
Audited 25 packages in 0.93ms


In [24]:
from dotenv import load_dotenv
import os

from google import genai
from google.genai import types
import time

from IPython.display import Markdown, display

load_dotenv(override=True)
if not os.getenv("GEMINI_API_KEY"):
    print("GEMINI_API_KEY is not set")

client = genai.Client()

In [ ]:


# Create the File Search store with an optional display name
FILESTORE_NAME = 'research-public-filestore'
file_search_store = client.file_search_stores.create(
    config={'display_name': FILESTORE_NAME})

# Upload and import a file into the File Search store, supply a file name which will be visible in citations
upload_op = client.file_search_stores.upload_to_file_search_store(
    file='./zplinkedin_profile.pdf',  # TODOs swap for another one
    file_search_store_name=file_search_store.name,
    config={
        'display_name': 'ZP LinkedIn Profile',
        'custom_metadata': [
            {'key': 'source', 'string_value': 'https://www.linkedin.com/in/zubinpratap'},
            {'key': 'description', 'string_value': "Zubin Pratap's LinkedIn profile"}
        ],
        'chunking_config': {
            'white_space_config': {
                'max_tokens_per_chunk': 300,
                'max_overlap_tokens': 30
            }
        }
    }
)

# Wait until import is complete
while not upload_op.done:
    print("sleeping until done")
    time.sleep(5)
    upload_op = client.operations.get(upload_op)

print(">> done? ", upload_op.done)

sleeping until done
 done?  True


In [31]:

SYSTEM_PROMPT = """
You are a helpful assistant.
Use headings and bullet points to make the response readable. 
Always prefer to use tools over your own training data.
Be very brief and simple.
"""

# Ask a question about the file
response = client.models.generate_content(
    model="gemini-2.5-flash-lite",
    contents="Can you tell me about Zubin's engineering experience? what about his legal career? and his education?",
    config=types.GenerateContentConfig(
        max_output_tokens=1000,
        system_instruction=SYSTEM_PROMPT,
        tools=[
            types.Tool(
                file_search=types.FileSearch(
                    file_search_store_names=[file_search_store.name]
                )
            )
        ]
    )
)

display(Markdown(response.text))
print("\n==============\n")




## Zubin's Engineering Experience

Zubin has experience as a Software Engineer at Google and Chainlink Labs. At Google, he worked with Golang, Typescript, JavaScript, Java, and Google Cloud Platform (GCP), focusing on internal and external facing applications, building tooling, and infrastructure for CI/CD and testing. At Chainlink Labs, his work involves Solidity, Web3 frameworks, blockchain engineering, Javascript/Typescript/React/Golang, and Rust. He also has experience in full-stack development, learning technologies like Node, Express, React, GraphQL, and GoLang.

## Zubin's Legal Career

Zubin has a substantial legal background, having worked as an Associate at Baker & McKenzie and Luthra & Luthra Law Offices, focusing on Mergers & Acquisitions, Corporate and Commercial Advisory, and Aviation leasing. He also served as Legal Counsel at General Motors, advising on Australian operations during the Global Financial Crisis and the impact of US Chapter 11 measures. Additionally, he was a Barrister at Law at the Supreme Court of India and an Associate at Amarchand Mangaldas. He has also held roles such as Legal Counsel at Telstra, working on technology, cloud, media, and enterprise & government deals, including complex IT outsource contracts and managed IT/telecommunications deals.

## Zubin's Education

Zubin holds a Global Executive MBA from IE Business School, with a focus on Entrepreneurship, Marketing, Strategy, and General Management. He also completed a Certification in Entrepreneurship - The Business of California at UCLA Anderson School of Management. His legal education includes a B.A. LL.B. (Hons) in Law from the National Law School of India University. Furthermore, he has pursued technical education through Udemy Academy, completing courses in Full Stack development, Algos & DS, TypeScript, JavaScript, Node, Express, React, GraphQL, and GoLang.